Author: Emily Wong \
January 22, 2023

# Resources

https://www.analyticsvidhya.com/blog/2018/06/comprehensive-guide-for-ensemble-models/#4

https://towardsdatascience.com/having-an-imbalanced-dataset-here-is-how-you-can-solve-it-1640568947eb

https://neptune.ai/blog/cross-validation-mistakes

# 1. Import libraries, methods, and data

## 1.1. Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Data wrangling
import pandas as pd
import numpy as np
from numpy.random import uniform, normal, seed

# Machine learning
import sklearn
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay, roc_auc_score, balanced_accuracy_score
from sklearn.model_selection import RepeatedKFold, RepeatedStratifiedKFold, RandomizedSearchCV, GridSearchCV, train_test_split
import scipy
from scipy.stats import randint
import xgboost as xgb
from imblearn.over_sampling import SMOTENC, RandomOverSampler, SMOTE
from imblearn.under_sampling import TomekLinks, NeighbourhoodCleaningRule, EditedNearestNeighbours, RandomUnderSampler
from imblearn.combine import SMOTEENN, SMOTETomek

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz
import matplotlib.pyplot as plt
import seaborn as sns # for kernel density plots

# for nested dictionary
import collections
def makehash():
    return collections.defaultdict(makehash)

# TPOT2
#from dask.distributed import Client, LocalCluster
#import tpot2

# Fairness
import aif360
import fairlearn
from fairlearn.metrics import demographic_parity_difference, demographic_parity_ratio, equalized_odds_difference, equalized_odds_ratio, false_negative_rate

The __demographic parity difference__ of 0 means that all groups have the same selection rate. For multiple groups, average across all pairwise differences. Ranges between 0 and 1.

The __demographic parity ratio__ of 1 means that all groups have the same selection rate.

The __equalized odds difference__ of 0 means that all groups have the same true positive, true negative, false positive, and false negative rates.

The __equalized odds ratio__ of 1 means that all groups have the same true positive, true negative, false positive, and false negative rates.

## 1.2 Reweighing Method

In [ ]:
def calc_weights(df, sens_features_name, outcome_name):
    ''' Calculate sample weights according to calculationg given in 
           F. Kamiran and T. Calders,  "Data Preprocessing Techniques for
           Classification without Discrimination," Knowledge and Information
           Systems, 2012.
    ''' 
    
    # combination of label and groups (outputs a table)
    sens_features = df[sens_features_name]
    outcome = df[outcome_name]
    tab = pd.DataFrame(pd.crosstab(index=sens_features, columns=outcome))

    # reweighing weights
    w = makehash()
    n = len(df)
    for r in tab.index:
        key1 = str(r)
        row_sum = tab.loc[r].sum(axis=0)
        for c in tab.columns:
            key2 = str(c)
            col_sum = tab[c].sum()
            if tab.loc[r,c] == 0:
                n_combo = 1
            else:
                n_combo = tab.loc[r,c]
            val = (row_sum*col_sum)/(n*n_combo)
            w[key1][key2] = val
    
    # Instance weights
    instance_weights = []
    for index, row in df.iterrows():
        race = row[sens_features_name]
        out = row[outcome_name]
        instance_weights.append(w[race][str(out)])

    return instance_weights

In [ ]:
def display_performance(X_train, y_train, X_test, y_test, model):
    # Train performance
    y_train_pred = model.predict(X_train)
    print("Train Accuracy:", np.round(accuracy_score(y_train, y_train_pred),5))
    print("Train Balanced Acc:",np.round(sklearn.metrics.balanced_accuracy_score(y_train, y_train_pred),5))
    #print("Train AUC:",np.round(sklearn.metrics.roc_auc_score(y_train, y_train_pred, multi_class='ovo'),5))
    #print("Train F1:",np.round(sklearn.metrics.f1_score(y_train, y_train_pred),5))
    cm = confusion_matrix(y_train, y_train_pred)
    print("Train Confusion Matrix:")
    print(cm)
    ConfusionMatrixDisplay(confusion_matrix=cm).plot(cmap=plt.cm.Greens);

    print("")

    # Test performance
    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    print("Test Accuracy:", np.round(test_accuracy,5))
    print("Test Balanced Acc:",np.round(sklearn.metrics.balanced_accuracy_score(y_test, y_pred),5))
    #print("Test AUC:",np.round(sklearn.metrics.roc_auc_score(y_test, y_pred, multi_class='ovo'),5))
    #print("Test F1:",np.round(sklearn.metrics.f1_score(y_test, y_pred),5))
    cm = confusion_matrix(y_test, y_pred)
    print("Test Confusion Matrix:")
    print(cm)
    ConfusionMatrixDisplay(confusion_matrix=cm).plot(cmap=plt.cm.Greens);

    print("")

    # Fairness metrics
    print("Equalized odds difference:",np.round(equalized_odds_difference(y_test, y_pred, sensitive_features=race_feature),2))

In [ ]:
def optim_thresh(X_test, y_test, model):
    # Find optimal threshold
    step_factor = 0.05 
    threshold_value = 0.05
    roc_score=0
    predicted_proba = model.predict_proba(X_test) #probability of prediction
    while threshold_value <=0.8: #continue to check best threshold upto probability 0.8
        temp_thresh = threshold_value
        predicted = (predicted_proba [:,1] >= temp_thresh).astype('int') #change the class boundary for prediction
        #print('Threshold',temp_thresh,'--',roc_auc_score(y_test, predicted))
        if roc_score<roc_auc_score(y_test, predicted, multi_class='ovo'): #store the threshold for best classification
            roc_score = roc_auc_score(y_test, predicted)
            thrsh_score = threshold_value
        threshold_value = threshold_value + step_factor
    print('---Optimum Threshold ---',np.round(thrsh_score,5),'--ROC--',np.round(roc_score,5))

    optim_thresh = thrsh_score
    y_pred_optim = (predicted_proba [:,1] >= optim_thresh).astype('int')
    print("Optimal Test Accuracy:",np.round(accuracy_score(y_test, y_pred_optim),5))
    print("Optimal Test Balanced Accuracy:",np.round(balanced_accuracy_score(y_test, y_pred_optim),5))
    print("Optimal Test AUC:",np.round(sklearn.metrics.roc_auc_score(y_test, y_pred_optim, multi_class='ovo'),5))

    # Create the confusion matrix
    cm = confusion_matrix(y_test, y_pred_optim)
    print("Test Confusion Matrix w/ Optimal Threshold:")
    print(cm)
    ConfusionMatrixDisplay(confusion_matrix=cm).plot(cmap=plt.cm.Greens);

    print("")

    print("Equalized odds difference w/ Optimal Threshold:",np.round(equalized_odds_difference(y_test, y_pred_optim,sensitive_features=race_feature),2))
    

## 1.3. Data

In [ ]:
all_data = pd.read_excel("Eynav cleaned data.xlsx")

In [ ]:
outcome = 'PHQ9_risk2'

data = all_data[['MOM_AGE','MOM_RACE','ETHNIC_GROUP','ZIP','MARITAL_STATUS','FINANCIAL_CLASS',
                 #'GRAVIDITY','PARITY','ABORTIONS',
                 #'CHILD_BIRTH_WT','CHILD_GESTATION_AGE_NUMBER',
                 'LBW','PTB',
                 'DELIVERY_METHOD','NICU_ADMIT','MFCU_ADMIT',
                 'PREE','GDM','GHTN',
                 'MOM_BMI','MOM_LOS','CHILD_LOS',
                 'HIST_ANXIETY','HIST_DEPRESS','HIST_BIPOLAR','HIST_OTHER',
                 'MED_PSYCH','MED_CARDIO',
                 outcome,'PHQ9_VALUE']]

## 1.3.3. Curate Data

In [ ]:
data = data.dropna()            # keep only complete data (for now)
data = data.sample(len(data))   # randomly shuffle rows
data.shape

In [ ]:
scale_data = data[['MOM_RACE','ETHNIC_GROUP','PHQ9_VALUE','PHQ9_risk2']]

In [ ]:
scale_data2 = scale_data[scale_data.PHQ9_risk2==1]

# create a grid 
g = sns.FacetGrid(scale_data2, col='MOM_RACE', hue='MOM_RACE', col_wrap=3)

# draw density plots
g = g.map(sns.kdeplot,"PHQ9_VALUE", cut=0, fill=True, common_norm=False, alpha=1, legend=False)

# control the title of each facet
g = g.set_titles("{col_name}")

# show the graph
#plt.show()

plt.savefig('Figure 1.png',dpi=350)

In [ ]:
data = data.drop(['PHQ9_VALUE'], axis=1)

In [ ]:
race = data['MOM_RACE']
ethnic = data['ETHNIC_GROUP']
out = data[outcome]

print("------------MEDIAN AGE------------")
print(pd.crosstab(index=race, columns=ethnic, values=data['MOM_AGE'], aggfunc=np.median))
print("Overall median age:",np.median(data[['MOM_AGE']]))

print("------------RACE/ETHNIC COUNTS------------")
print(pd.DataFrame(pd.crosstab(index=race, columns=ethnic)))

print("------------RACE/ETHNIC PMAD------------")
print(pd.crosstab(index=[ethnic,race], columns=out, normalize='index'))

print("Overall PMAD:",np.mean(data[[outcome]]))

In [ ]:
# binary-class
count0, count1 = data[outcome].value_counts()
print(count0, count1)

x = ['0','1']
y = [count0, count1]
plt.bar(x, y)

In [ ]:
print("N:",data.shape)

## 1.3.4. Weight Data

In [ ]:
data['w'] = calc_weights(df=data, sens_features_name="MOM_RACE", outcome_name=outcome)

## 1.3.5. Get Dummies and Split

In [ ]:
# get dummy variables
data = pd.get_dummies(data)

Split data. Can specify whether to use stratify sampling or not.

In [ ]:
# split into X and y
X = data.drop([outcome], axis=1)
y = data[[outcome]]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.85, test_size=0.15, shuffle=True, stratify=y, random_state=0)
X_test = X_test.drop(['w'], axis=1)

# Sensitive features
race_feature = X_test[['MOM_RACE_Asian or Native Hawaiian or Other Pacific Islander',
                       'MOM_RACE_Black or African American',
                       'MOM_RACE_Multiracial',
                       'MOM_RACE_Other',
                       'MOM_RACE_Unknown',
                       'MOM_RACE_White',
                       'MOM_RACE_Hispanic White']]

In [ ]:
# binary-class
count0_train, count1_train = y_train.value_counts()
print(count0_train, count1_train)

count0_test, count1_test = y_test.value_counts()
print(count0_test, count1_test)

# 2. Handle imbalanced data

## 2.1. Simple Over Sampling Minority (PMAD)

In [ ]:
ros = RandomOverSampler(sampling_strategy = "auto",random_state=0)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)
weights_ros = X_train_ros['w']
X_train_ros = X_train_ros.drop(['w'], axis=1)
y_train_ros.value_counts()

## 2.2. Simple Under Sampling Majority (PMAD)

In [ ]:
rus = RandomUnderSampler(sampling_strategy = "auto", random_state=0)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)
weights_rus = X_train_rus['w']
X_train_rus = X_train_rus.drop(['w'], axis=1)
y_train_rus.value_counts()

# 3. Modeling

In [ ]:
# Extract weights and drop from training and test data frames
weights = X_train['w']
X_train = X_train.drop(['w'], axis=1)

## XG Boost Classifier

https://www.kaggle.com/code/stuarthallows/using-xgboost-with-scikit-learn

### Finetune XG Boost Classifier Through Randomized Search CV

In [ ]:
x = X_train_rus
y = y_train_rus.values.ravel()
w = weights_rus

xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=0)

params = {"colsample_bytree": [0.3, 0.4, 0.5, 0.6, 0.7],
          "gamma": [0, 0.1, 0.2, 0.3, 0.4, 0.5],
          "learning_rate": [0.01, 0.05, 0.1, 0.2,  0.3], # default 0.1
          "n_estimators": list(range(100,150+1)), # 100 to 150
          "subsample": [0.4,0.5,0.6],
          "alpha":[0,5,10,25,50,75,100],
          "lambda": [0,5,10,25,50,75,100]
          }

rkf = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=0)
rand_search = RandomizedSearchCV(xgb_model, 
                                 param_distributions=params, 
                                 random_state=42, 
                                 n_iter=10, 
                                 cv=rkf, 
                                 verbose=1, 
                                 n_jobs=1, 
                                 return_train_score=True,
                                 scoring = ['balanced_accuracy','f1','roc_auc'],
                                 refit = 'roc_auc')

rand_search.fit(x, y,sample_weight=w)

In [ ]:
best_xgb = rand_search.best_estimator_
print('XGB  best hyperparameters:',  rand_search.best_params_)

display_performance(X_train=x, y_train=y, X_test=X_test, y_test=y_test, model=best_xgb)

In [ ]:
optim_thresh(X_test=X_test, y_test=y_test, model=best_xgb)

### Finetuned XG Boost Classifier

XGB  best hyperparameters: {'subsample': 0.4, 'n_estimators': 136, 'learning_rate': 0.01, 'lambda': 50, 'gamma': 0.2, 'colsample_bytree': 0.6, 'alpha': 0}

---Optimum Threshold --- 0.50

In [ ]:
x = X_train_rus
y = y_train_rus.values.ravel()
w = weights_rus

xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=0, subsample=0.4, n_estimators=136, 
                              learning_rate=0.01, reg_lambda=50, gamma=0.2, colsample_bytree=0.6, alpha=0)
xgb_model.fit(x,y,sample_weight=w)
optim_thresh(X_test=X_test, y_test=y_test, model=xgb_model)

## Balanced Bagging Classifer

### Finetune Balanced Bagging Classifier Through Randomized Search CV

In [ ]:
x = X_train
y = y_train.values.ravel()
w = weights


bbc = BalancedBaggingClassifier(random_state=0, 
                                sampling_strategy='auto',
                                base_estimator=xgb_model)
parameter_space = {'max_features': [0.9,0.95,0.96,0.97,0.98,0.99,1.0],
                   'n_estimators': list(range(5,15+1)) # 5 to 15
                  }

# Use random search to find the best hyperparameters
rkf = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=0)
rand_search = RandomizedSearchCV(bbc, 
                                 param_distributions = parameter_space,
                                 random_state=42,
                                 n_iter=10, 
                                 cv=rkf,
                                 verbose=1,
                                 n_jobs=1, 
                                 return_train_score=True,
                                 scoring = ['balanced_accuracy','f1','roc_auc'],
                                 refit = 'roc_auc')

# Fit the random search object to the data
rand_search.fit(X_train, y_train.values.ravel())

In [ ]:
best_bbc = rand_search.best_estimator_
print('Ensemble best hyperparameters:',  rand_search.best_params_)

display_performance(X_train=x, y_train=y, X_test=X_test, y_test=y_test, model=best_bbc)

optim_thresh(X_test=X_test, y_test=y_test, model=best_bbc)

### Finetuned Balanced Bagging Classifier

Ensemble best hyperparameters: {'n_estimators': 9, 'max_features': 0.9}

In [ ]:
x = X_train
y = y_train.values.ravel()
w = weights

bbc_model = BalancedBaggingClassifier(random_state=0,sampling_strategy='auto',base_estimator=xgb_model,
                                      n_estimators=9, max_features=0.9)
bbc_model.fit(x,y)
optim_thresh(X_test=X_test, y_test=y_test, model=bbc_model)

In [ ]:
y_pred_orig = bbc_model.predict_proba(X_test)
y_pred_optim = (y_pred_orig [:,1] >= 0.50).astype('int') #change the class boundary for prediction

from fairlearn.metrics import MetricFrame, false_negative_rate
mf = MetricFrame(metrics={'fnr': false_negative_rate},
                y_true = y_test,
                y_pred = y_pred_optim,
                sensitive_features=race_feature)
print(mf.by_group) # what is this output?
#print(mf.difference())

In [ ]:
races = ['MOM_RACE_Asian or Native Hawaiian or Other Pacific Islander',
         'MOM_RACE_Black or African American',
         'MOM_RACE_Hispanic White',
         'MOM_RACE_Multiracial',
         'MOM_RACE_Other',
         'MOM_RACE_Unknown',
         'MOM_RACE_White']

y_pred = best_bbc.predict(X_test)
optim_threshold = 0.5
y_pred_optim = (best_bbc.predict_proba(X_test)[:,1] >= optim_threshold).astype('int')
for r in races:
    feature = X_test[[r]]
    measure = np.round(equalized_odds_difference(y_test, y_pred, sensitive_features=feature),5)
    print("Equalized odds difference:",r,":",measure)

## Logistic Regression

### Finetune Logistic Regression Through Randomized Search CV

In [ ]:
x = X_train_rus
y = y_train_rus.values.ravel()
w = weights_rus

glm = LogisticRegression(max_iter=1000, solver='liblinear')

seed(0)
parameter_space = {'class_weight': [{0:1, 1:2},{0:1, 1:3},{0:1, 1:5},{0:1, 1:5},{0:1, 1:6},{0:1, 1:7},{0:1, 1:8},{0:1, 1:9},{0:1, 1:10},
                                    {0:2, 1:2},{0:2, 1:3},{0:2, 1:5},{0:2, 1:5},{0:2, 1:6},{0:2, 1:7},{0:2, 1:8},{0:2, 1:9},{0:2, 1:10},
                                    {0:3, 1:2},{0:3, 1:3},{0:3, 1:5},{0:3, 1:5},{0:3, 1:6},{0:3, 1:7},{0:3, 1:8},{0:3, 1:9},{0:3, 1:10}]
                  }

# Use random search to find the best hyperparameters
rkf = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=0)
rand_search = RandomizedSearchCV(glm, 
                                 param_distributions = parameter_space, 
                                 random_state=42,
                                 n_iter=10, 
                                 cv=rkf,
                                 verbose=1,
                                 scoring = ['balanced_accuracy','f1','roc_auc'],
                                 refit = 'roc_auc')

# Fit the random search object to the data
rand_search.fit(x, y, sample_weight=w)

In [ ]:
# Print the best hyperparameters
best_glm = rand_search.best_estimator_
print('Best GLM hyperparameters:',  rand_search.best_params_)

display_performance(X_train=x, y_train=y, X_test=X_test, y_test=y_test, model=best_glm)

In [ ]:
optim_thresh(X_test=X_test, y_test=y_test, model=best_glm)

### Finetuned Logistic Regression

In [ ]:
x = X_train_rus
y = y_train_rus.values.ravel()
w = weights_rus

glm_model = LogisticRegression(max_iter=1000, solver='liblinear',class_weight= {0: 1, 1: 2},random_state=0)
glm_model.fit(x, y, sample_weight=w)
optim_thresh(X_test=X_test, y_test=y_test, model=glm_model)

## Random Forest

### Finetune Random Forest Through Randomized Search CV

In [ ]:
x = X_train_rus
y = y_train_rus.values.ravel()
w = weights_rus

rf = RandomForestClassifier(max_features='sqrt',random_state=0)

seed(1)
parameter_space = {'min_samples_split':np.round(uniform(1,50,20),0).astype('int64'),
                   'min_samples_leaf':np.round(uniform(1,50,20),0).astype('int64'),
                   'n_estimators': np.round(uniform(50,500,20),0).astype('int64'),
                   'max_depth': np.round(uniform(1,20,20),0).astype('int64')
                  }

# Use random search to find the best hyperparameters
rkf = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=0)
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = parameter_space, 
                                 n_iter=10, 
                                 cv=rkf,
                                 verbose=1,
                                 random_state=42,
                                 scoring = ['balanced_accuracy','f1','roc_auc'],
                                 refit = 'roc_auc')

# Fit the random search object to the data
rand_search.fit(x, y, sample_weight=w)

In [ ]:
best_rf = rand_search.best_estimator_
print('RF Best hyperparameters:',  rand_search.best_params_)

display_performance(X_train=x, y_train=y, X_test=X_test, y_test=y_test, model=best_rf)

In [ ]:
optim_thresh(X_test=X_test, y_test=y_test, model=best_rf)

### Finetuned Random Forest

RF Best hyperparameters: {'n_estimators': 271, 'min_samples_split': 8, 'min_samples_leaf': 38, 'max_depth': 4}

In [ ]:
x = X_train_rus
y = y_train_rus.values.ravel()
w = weights_rus

rf_model = RandomForestClassifier(max_features='sqrt', n_estimators=271, min_samples_split=8,
                                  min_samples_leaf=38, max_depth=4, random_state=0)
rf_model.fit(x, y, sample_weight=w)
optim_thresh(X_test=X_test, y_test=y_test, model=rf_model)

In [ ]:
# all races together
from fairlearn.metrics import MetricFrame, true_positive_rate, false_positive_rate
mf = MetricFrame(metrics={'tpr' : true_positive_rate,
                          'fpr' : false_positive_rate,
                         },
                 y_true = y_test,
                 y_pred = y_pred,
                 sensitive_features=race_feature,
                )
print(mf.by_group)
print(mf.difference())

In [ ]:
# only one race
mf = MetricFrame(metrics={'tpr' : true_positive_rate,
                          'fpr' : false_positive_rate,
                          'fnr' : false_negative_rate
                         },
                 y_true = y_test,
                 y_pred = y_pred,
                 sensitive_features=race_feature['MOM_RACE_Hispanic White'],
                )
print(mf.by_group)
print(mf.difference())

In [ ]:
races = ['MOM_RACE_Asian or Native Hawaiian or Other Pacific Islander',
         'MOM_RACE_Black or African American',
         'MOM_RACE_Latinx',
         'MOM_RACE_Multiracial',
         'MOM_RACE_Other',
         'MOM_RACE_Unknown',
         'MOM_RACE_White']

y_pred = best_rf.predict(X_test)
optim_threshold = 0.5
y_pred_optim = (best_rf.predict_proba(X_test)[:,1] >= optim_threshold).astype('int')
for r in races:
    feature = X_test[[r]]
    measure = np.round(equalized_odds_difference(y_test, y_pred, sensitive_features=feature),5)
    print("Equalized odds difference:",r,":",measure)

In [ ]:
# save this file and output as html
import os
os.system('jupyter nbconvert --to html data_analysis.ipynb')